In [1]:

import numpy as np
print(np.__version__)

from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, WhooshChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking, SemanticChunking
import os
import pickle
import json

1.26.4


c:\Users\User\Desktop\Диплом\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedder = HuggingFaceEmbedder(model_name='BAAI/bge-m3', normalizer=True)

import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('../data_test')
chunks = WhooshChunkStorage(scoring_function="BM25F", normalizer=True)
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)


algorithm = SemanticChunking(raw_storage=documents, chunk_storage=chunks, embedder=embedder, threshold=0.5, max_chunk_size=1000)
algorithm.execute(visualize=True)

from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = FaissIndexFlatL2(dimensionality=embedder.get_dimensionality())
embedder.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)


Splitting text into the sentences


28it [00:00, 13996.01it/s]


creating embeddings of the sentences for semantic grouping


100%|██████████| 28/28 [00:10<00:00,  2.77it/s]


Adding chunks to the storage


100%|██████████| 18/18 [00:00<00:00, 90.44it/s]


Converting ChunkStorage to VectorIndex


100%|██████████| 18/18 [00:08<00:00,  2.06it/s]


In [5]:
from RelFunc import *
from RankMetr import RankMetr

    
    
class Benchmark:
    def __init__(self, querinator, embedenator, RelFunc: RelevanceFunction, rank_metr: RankMetr, top_k=5, bench_path: os.PathLike = '../benchmark/QnA/'):
        self.querinator = querinator
        self.embedenator = embedenator
        self.benchmarks = []
        self._get_benchmark(bench_path)
        self.top_k = top_k
        self.RF = RelFunc
        self.RF.set_embedder(embedenator)
        self.rank_metr = rank_metr  # Store the rank_metr instance
    
    def _get_benchmark(self, path):
        for p in os.listdir(path):
            with open(path+p, 'r', encoding='utf-8') as bench:
                self.benchmarks.append(json.load(bench))
                
    def run_benchmark(self):
        for bench_num, b in enumerate(self.benchmarks):
            
            chunk_real = b['ideal_context']        
            self.RF.set_ideal_context(chunk_real)
                
            chunk_model = self.get_chunks(b['question'])
            
            rel_array = []
            for i in range(self.top_k):
                rel_array.append(self.RF.get_relevance(chunk_model[i]))
            
            print('-'*20)
            print(f'Benchmark number: {bench_num}')
            for func_name in dir(self.rank_metr):  # Use dir to get all attributes of rank_metr
                if func_name.startswith('_'):  # Skip private methods
                    continue
                func = getattr(self.rank_metr, func_name)
                if callable(func):  # Check if the attribute is a callable method
                    print(f'{func_name}: {func(rel_array)}')
    
    def get_chunks(self, query: str):
        query_vector = self.embedenator.embed(text=query)
        answer = self.querinator.query(query_vector, top_k=self.top_k)
        return [chunks[element.chunk_storage_key] for element in answer]
    
 
bench = Benchmark(querinator=vector_index,
                  embedenator=embedder,
                  RelFunc=IoU(0.2),
                  rank_metr=RankMetr(),
                  top_k=5)
bench.run_benchmark()

--------------------
Benchmark number: 0
ap: 0.8333333333333333
erp: 0.359336017285326
ndcg: 0.9197207891481876
--------------------
Benchmark number: 1
ap: 1.0
erp: 0.46538892508553426
ndcg: 1.0
--------------------
Benchmark number: 2
ap: 1.0
erp: 0.46020393390890024
ndcg: 1.0
--------------------
Benchmark number: 3
ap: 0.0
erp: 0.19588061106658472
ndcg: 0.0
--------------------
Benchmark number: 4
ap: 1.0
erp: 0.35755315246070896
ndcg: 1.0
--------------------
Benchmark number: 5
ap: 0.5
erp: 0.19656266468092845
ndcg: 0.6309297535714575
--------------------
Benchmark number: 6
ap: 0.5
erp: 0.2505108647878942
ndcg: 0.6309297535714575
--------------------
Benchmark number: 7
ap: 0.25
erp: 0.2496457610980466
ndcg: 0.43067655807339306
--------------------
Benchmark number: 8
ap: 0.3666666666666667
erp: 0.32351703043500274
ndcg: 0.5437713091520254
--------------------
Benchmark number: 9
ap: 1.0
erp: 0.5097234677658711
ndcg: 1.0
--------------------
Benchmark number: 10
ap: 0.95
erp: 0.